# 自动写作

## 文本预处理

In [2]:
# 载入所需工具包
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.
d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\py

In [44]:
df = pd.read_csv(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\金庸-射雕英雄传txt精校版.txt", names=["txt"], sep='aaa', encoding="gbk")

# 构建新列
print(len(df))
print(df.shape)
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

df["head"] = df.txt.apply(m_mid)
df["mid"] = df.txt.apply(m_head)
df["len"] = df.txt.apply(len)

# 标明章序号
chapnum = 0
for i in range(len(df)):
    if df["mid"][i] == "第" and df["head"][i] > 0 and df["len"][i] < 30:
        chapnum += 1
    if chapnum >= 40 and df.txt[i] == "附录一：成吉思汗家族":
        chapnum = 0
    df.loc[i, "chap"] = chapnum

dfgrp = df.groupby("chap")
chapter = dfgrp.apply(sum)
chapter = chapter[chapter.index != 0]
chapter.head()

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


7165
(7165, 1)


,txt,head,mid,len,chap
chap,,,,,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0


In [8]:
rawtxt = pd.read_csv(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\r&j.txt",
                     sep = 'aaaaa', names = ['txt'], engine = 'python', encoding="utf-8")
rawtxt.head()

,txt
0,This Etext file is presented by Project Gutenb...
1,"cooperation with World Library, Inc., from the..."
2,Future and Shakespeare CDROMS. Project Gutenb...
3,Etexts that are NOT placed in the Public Domain!!
4,*This Etext has certain copyright implications...


In [10]:
rawtxt.txt[1]

'cooperation with World Library, Inc., from their Library of the'

In [11]:
# 处理大小写，末尾增加空格
def m_perproc(tmpstr):
    return (tmpstr + " ").lower()

rawtxt.txt = rawtxt.txt.apply(m_perproc)
rawtxt.txt[1]

'cooperation with world library, inc., from their library of the '

In [12]:
raw_txt = rawtxt.txt.apply("sum")
raw_txt

'this etext file is presented by project gutenberg, in cooperation with world library, inc., from their library of the future and shakespeare cdroms.  project gutenberg often releases etexts that are not placed in the public domain!! *this etext has certain copyright implications you should read!* <<this electronic version of the complete works of william shakespeare is copyright 1990-1993 by world library, inc., and is provided by project gutenberg with permission.  electronic and machine readable copies may be distributed so long as such copies (1) are for your or others personal use only, and (2) are not distributed or used commercially.  prohibited commercial distribution includes by any service that charges for download time or for membership.>> *project gutenberg is proud to cooperate with the world library* in the presentation of the complete works of william shakespeare for your reading for education and entertainment.  however, this is neither shareware nor public domain. . .a

In [13]:
# 将字符转换为数值代码以便处理
chars = sorted(list(set(raw_txt))) # 生成字符list
char_to_int = dict((c, i) for i, c in enumerate(chars)) # 字符-数值对应字典
int_to_char = dict((i, c) for i, c in enumerate(chars)) # 数值-字符对应字典
chars

[' ',
 '!',
 '"',
 '#',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~']

## 构造训练测试集

In [14]:
seq_length = 100
x = []; y = []
for i in range(0, len(raw_txt) - seq_length):
    given = raw_txt[i:i + seq_length] # 将前seq_length个字符作为预测用变量
    predict = raw_txt[i + seq_length] # 将当前字符作为因变量
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])

In [15]:
x[:3]

[[52,
  40,
  41,
  51,
  0,
  37,
  52,
  37,
  56,
  52,
  0,
  38,
  41,
  44,
  37,
  0,
  41,
  51,
  0,
  48,
  50,
  37,
  51,
  37,
  46,
  52,
  37,
  36,
  0,
  34,
  57,
  0,
  48,
  50,
  47,
  42,
  37,
  35,
  52,
  0,
  39,
  53,
  52,
  37,
  46,
  34,
  37,
  50,
  39,
  9,
  0,
  41,
  46,
  0,
  35,
  47,
  47,
  48,
  37,
  50,
  33,
  52,
  41,
  47,
  46,
  0,
  55,
  41,
  52,
  40,
  0,
  55,
  47,
  50,
  44,
  36,
  0,
  44,
  41,
  34,
  50,
  33,
  50,
  57,
  9,
  0,
  41,
  46,
  35,
  11,
  9,
  0,
  38,
  50,
  47,
  45,
  0,
  52,
  40,
  37],
 [40,
  41,
  51,
  0,
  37,
  52,
  37,
  56,
  52,
  0,
  38,
  41,
  44,
  37,
  0,
  41,
  51,
  0,
  48,
  50,
  37,
  51,
  37,
  46,
  52,
  37,
  36,
  0,
  34,
  57,
  0,
  48,
  50,
  47,
  42,
  37,
  35,
  52,
  0,
  39,
  53,
  52,
  37,
  46,
  34,
  37,
  50,
  39,
  9,
  0,
  41,
  46,
  0,
  35,
  47,
  47,
  48,
  37,
  50,
  33,
  52,
  41,
  47,
  46,
  0,
  55,
  41,
  52,
  40,
  0,
  55,
  4

In [16]:
y[:3]


[41, 50, 0]

In [17]:
n_patterns = len(x)
n_vocab = len(chars)

# 把x变成LSTM需要的格式，reshape最后的1表示每个数值均为单独一个向量（代表一个字母输入）
x = np.reshape(x, (n_patterns, seq_length, 1)) 
x = x / float(n_vocab) # 转换为0-1之间的数值以方便计算
x[0]

array([[0.86666667],
       [0.66666667],
       [0.68333333],
       [0.85      ],
       [0.        ],
       [0.61666667],
       [0.86666667],
       [0.61666667],
       [0.93333333],
       [0.86666667],
       [0.        ],
       [0.63333333],
       [0.68333333],
       [0.73333333],
       [0.61666667],
       [0.        ],
       [0.68333333],
       [0.85      ],
       [0.        ],
       [0.8       ],
       [0.83333333],
       [0.61666667],
       [0.85      ],
       [0.61666667],
       [0.76666667],
       [0.86666667],
       [0.61666667],
       [0.6       ],
       [0.        ],
       [0.56666667],
       [0.95      ],
       [0.        ],
       [0.8       ],
       [0.83333333],
       [0.78333333],
       [0.7       ],
       [0.61666667],
       [0.58333333],
       [0.86666667],
       [0.        ],
       [0.65      ],
       [0.88333333],
       [0.86666667],
       [0.61666667],
       [0.76666667],
       [0.56666667],
       [0.61666667],
       [0.833

In [18]:
# 将因变量的类型正确指定为类别
y = np_utils.to_categorical(y) 
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## 建立LSTM模型

In [19]:
model = Sequential() 
# LSTM层指定为128个神经元
model.add(LSTM(128, input_shape = (x.shape[1], x.shape[2]))) 
model.add(Dropout(0.2)) # 抛弃20%的结果，防止过拟合
model.add(Dense(y.shape[1], activation = 'softmax')) # 使用标准的NN作为内核
# 指定损失函数
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam') 

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
# batch_size为分批量将数据用于训练，以减小计算资源的需求
# epochs次数越多，模型训练效果越好，但所需时间也线性增加
model.fit(x, y, epochs = 2, batch_size = 64) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
149672/149672 [==============================] - 197s 1ms/step - loss: 2.9230
Epoch 2/2
149672/149672 [==============================] - 178s 1ms/step - loss: 2.8044


## 进行文本预测

In [36]:
def predict_next(input_array): # 进行下一个字符的预测
    x = np.reshape([0 for i in range(seq_length - len(input_array))]
                   + input_array, (1, seq_length, 1)) # 生成预测用的x序列
    x = x / float(n_vocab)
    y = model.predict(x)
    return y

def string_to_index(raw_input): # 将输入的字符转换为索引值
    res = []
    for c in raw_input[(len(raw_input) - seq_length):]:
        res.append(char_to_int[c])
    return res

def y_to_char(y): # 将预测结果由索引值转换回字符
    largest_index = y.argmax() # 取最大数值对应的索引值
    c = int_to_char[largest_index]
    return c

In [37]:
def generate_article(init, rounds = 50): # 按照指定的字符长度进行预测
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string += n # 将预测到的新字符合并，用于下一步预测
    return in_string

In [38]:
# 进行字母预测
init = 'We produce about two million dollars for each hour we work. The fifty hours is one conservative estimate for how long'
article = generate_article(init)
article

'we produce about two million dollars for each hour we work. the fifty hours is one conservative estimate for how long that sha toa what io the wound oo the toan the to'

# 将LSTM与word2vec结合实现中文自动写作

In [40]:
# 载入所需工具包
import jieba
from gensim.models.word2vec import Word2Vec

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [45]:
jinyong_words_path = r'D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\金庸小说词库.txt'
jieba.load_userdict(jinyong_words_path) # dict为自定义词典的路径

# 以整句或者整段为基本单位进行分析显然更为合适
corpus = [jieba.lcut(item) for item in df.txt]
corpus[:3]

[['全本',
  '全集',
  '精校',
  '小说',
  '尽',
  '在',
  '：',
  'http',
  ':',
  '/',
  '/',
  'www',
  '.',
  'yimuhe',
  '.',
  'com',
  '/',
  'u',
  '/',
  'anglewing2620',
  '-',
  '1',
  '.',
  'html'],
 ['更',
  '多',
  '资源',
  '下载',
  '：',
  'http',
  ':',
  '/',
  '/',
  'qqzone',
  '.',
  'ctdisk',
  '.',
  'com'],
 ['※',
  '声明',
  '：',
  '本',
  '电子书',
  '仅供',
  '读者',
  '预览',
  ',',
  '请',
  '在',
  '下载',
  '24',
  '小时',
  '内',
  '删除',
  '，',
  '不得',
  '用作',
  '商业用途',
  '；',
  '如果',
  '喜欢',
  '请',
  '购买',
  '正版',
  '图书',
  '！',
  '※']]

In [46]:
# 此处完全可以使用外部语料库进行更全面的训练
w2v_model = Word2Vec(corpus, size = 100, window = 5, 
                     min_count = 5, workers = 4)

In [47]:
w2v_model.wv['郭啸天']

array([-0.01997127, -0.1608803 , -0.28723073, -0.4448491 , -0.22112069,
       -0.09610055,  0.23912182, -0.31313825, -0.03963013, -0.15955736,
       -0.13434343, -0.30339324,  0.19292244, -0.30573782, -0.20047355,
        0.45574093, -0.04386814, -0.03229786, -0.22158848, -0.14827605,
        0.11687801,  0.00747179, -0.07738932, -0.18555674, -0.31863645,
       -0.08753093,  0.5019673 ,  0.06627637,  0.3347754 , -0.06323647,
       -0.28750482, -0.12433083,  0.30417618, -0.03966825, -0.33538708,
        0.01587437, -0.4660802 , -0.29275146,  0.16366668, -0.32376808,
       -0.26230386,  0.4117414 , -0.0523882 , -0.22532773, -0.2549575 ,
       -0.19077791, -0.20056604,  0.03690925, -0.06785063, -0.30534375,
       -0.51505405, -0.28568837, -0.1312137 , -0.15654978,  0.26516065,
        0.03677496, -0.19038089, -0.01009627,  0.10734184, -0.21143956,
       -0.17847787, -0.16178377, -0.05782022,  0.2272531 , -0.15335672,
       -0.10196341, -0.10179174, -0.25192964,  0.40604272, -0.13

In [48]:
# 将数据还原为一个长list
raw_input = [item for sublist in corpus for item in sublist]
print(len(raw_input))
raw_input[:10]

619643


['全本', '全集', '精校', '小说', '尽', '在', '：', 'http', ':', '/']

In [49]:
# 列出模型中纳入的词条
vocab = w2v_model.wv.vocab
vocab

{'额伦夫人': <gensim.models.keyedvectors.Vocab at 0x1d1e56f7b70>,
 '涌出': <gensim.models.keyedvectors.Vocab at 0x1d183006438>,
 '一说': <gensim.models.keyedvectors.Vocab at 0x1d195305828>,
 '统兵': <gensim.models.keyedvectors.Vocab at 0x1d18300a4a8>,
 '梅师姊': <gensim.models.keyedvectors.Vocab at 0x1d195338f60>,
 '疼痛': <gensim.models.keyedvectors.Vocab at 0x1d18300f080>,
 '贼': <gensim.models.keyedvectors.Vocab at 0x1d1953665c0>,
 '落后': <gensim.models.keyedvectors.Vocab at 0x1d19537b6d8>,
 '后': <gensim.models.keyedvectors.Vocab at 0x1d195305860>,
 '杀声': <gensim.models.keyedvectors.Vocab at 0x1d1e56eae80>,
 '发招': <gensim.models.keyedvectors.Vocab at 0x1d18300e940>,
 '荒山': <gensim.models.keyedvectors.Vocab at 0x1d1952d42b0>,
 '奇幻': <gensim.models.keyedvectors.Vocab at 0x1d19537b710>,
 '没出息': <gensim.models.keyedvectors.Vocab at 0x1d19537e128>,
 '所长': <gensim.models.keyedvectors.Vocab at 0x1d19531e940>,
 '约会': <gensim.models.keyedvectors.Vocab at 0x1d18300f0b8>,
 '银两': <gensim.models.keyedvectors.Voc

In [50]:
# min_count = 5参数会过滤掉低频词，因此需要在文本中同步清除这些低频词
text_stream = []

for word in raw_input:
    if word in vocab:
        text_stream.append(word)

print(len(text_stream))
text_stream[:10]

558799


['小说', '尽', '在', '：', '/', '/', '.', '.', '/', '/']

## 构造训练集和测试集

In [51]:
seq_length = 10 # 取前面10个单词用于预测
x = []; y = []
for i in range(0, len(text_stream) - seq_length):
    given = text_stream[i : i + seq_length]
    predict = text_stream[i + seq_length]
    x.append(np.array([w2v_model.wv[word] for word in given]))
    y.append(w2v_model.wv[predict])

In [52]:
len(x)

558789

In [53]:
x[0][0]

array([ 0.049283  ,  0.03380921, -0.35554603, -0.36246535, -0.3519603 ,
        0.25502715,  0.3337215 , -0.5055511 ,  0.10683467, -0.05806661,
        0.04317866, -0.46598485,  0.23497795,  0.00925067, -0.07761536,
        0.05777869, -0.50303966, -0.45923528, -0.45032433, -0.31640914,
        0.17991441, -0.26316983, -0.10847148, -0.25017306, -0.45811749,
       -0.3210428 ,  0.55954784,  0.30229008,  0.22738396,  0.21401812,
        0.00723053, -0.1383395 , -0.02209738, -0.04840301, -0.66360176,
        0.02091651, -0.40684316, -0.50688577,  0.40212214, -0.24411422,
       -0.25062916,  0.23879844,  0.08279792,  0.1285475 ,  0.3244009 ,
       -0.1493984 , -0.16343482, -0.22707872, -0.01762   , -0.09963102,
       -0.1664938 ,  0.17540902, -0.38015258, -0.13895833,  0.21384548,
        0.19112782, -0.15787123,  0.18954688,  0.01917979,  0.08300259,
       -0.22550246, -0.65263   ,  0.22211748, -0.01142071, -0.44375175,
       -0.2868831 , -0.19405067, -0.9339679 ,  0.22207105,  0.06

In [54]:
y[0]

array([-0.08383094, -0.17119533, -1.3561269 , -0.30238363, -0.5379825 ,
        0.597368  ,  0.78167814, -1.2629952 ,  0.64624876, -0.06554709,
        0.21199688, -1.1940835 ,  0.27952144,  0.26164073, -0.39651406,
       -0.82904744, -0.37760937, -1.0411144 , -1.4300909 , -0.3075369 ,
        0.4659976 , -0.6507042 , -0.04639957, -0.21707013, -0.48621255,
       -0.31010112,  1.7795377 ,  1.1779109 ,  0.19783665,  0.5007072 ,
        0.2009324 ,  0.00510134, -0.40014625, -0.5261832 , -0.46161267,
        0.28435874, -0.41976237, -0.63782835,  0.4781225 ,  0.01229712,
       -0.5639564 ,  0.31439224,  0.25762075,  0.3829558 ,  0.82503384,
       -0.08279676,  0.10897397, -0.98335713,  0.04197844, -0.33008805,
       -0.1658148 ,  0.62398636, -0.7410817 , -0.69185305,  0.36287197,
        0.2618282 ,  0.5088761 ,  0.46224278,  0.12688567,  0.5427201 ,
       -0.07906884, -1.413587  ,  0.3171477 , -0.58502877, -1.1616849 ,
       -1.0619206 , -0.1973948 , -1.9644413 ,  0.26221576,  1.12

In [55]:
x = np.reshape(x, (-1, seq_length, 100)) # 每一个词条，对应一个word2vec向量
y = np.reshape(y, (-1, 100))

## 建立LSTM模型

In [56]:
model = Sequential()
model.add(LSTM(128, input_shape = (seq_length, 100)))
model.add(Dropout(0.2))
model.add(Dense(100, activation = 'sigmoid'))
model.compile(loss = 'mse', optimizer = 'adam')

In [57]:
model.fit(x, y, epochs = 5, batch_size = 64)

Epoch 1/5
558789/558789 [==============================] - 89s 159us/step - loss: 0.2913
Epoch 2/5
558789/558789 [==============================] - 86s 154us/step - loss: 0.2852
Epoch 3/5
558789/558789 [==============================] - 96s 172us/step - loss: 0.2839
Epoch 4/5
558789/558789 [==============================] - 95s 169us/step - loss: 0.2832
Epoch 5/5
558789/558789 [==============================] - 89s 160us/step - loss: 0.2826


In [58]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12900     
Total params: 130,148
Trainable params: 130,148
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.save_weights('LSTM.h5') # 文件类型是HDF5

In [60]:
model.load_weights('LSTM.h5')

In [61]:
model.fit(x, y, epochs = 10) # 按照指定的数据和参数继续训练模型

Epoch 1/10
558789/558789 [==============================] - 126s 225us/step - loss: 0.2825
Epoch 2/10
558789/558789 [==============================] - 127s 228us/step - loss: 0.2821
Epoch 3/10
558789/558789 [==============================] - 122s 218us/step - loss: 0.2818
Epoch 4/10
558789/558789 [==============================] - 138s 248us/step - loss: 0.2816
Epoch 5/10
558789/558789 [==============================] - 129s 232us/step - loss: 0.2813
Epoch 6/10
558789/558789 [==============================] - 122s 219us/step - loss: 0.2811
Epoch 7/10
558789/558789 [==============================] - 124s 223us/step - loss: 0.2810
Epoch 8/10
558789/558789 [==============================] - 123s 220us/step - loss: 0.2808
Epoch 9/10
558789/558789 [==============================] - 121s 217us/step - loss: 0.2807
Epoch 10/10
558789/558789 [==============================] - 122s 219us/step - loss: 0.2805


In [62]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'LSTM_best.hdf5', 
                               monitor = 'val_loss', 
                               save_best_only = True, 
                               verbose = 1)

In [63]:
model.compile(loss = 'mse', optimizer = 'adam')

In [64]:
model.fit(x, y, epochs = 50, 
          validation_data = (x, y), 
          callbacks = [checkpointer])

Train on 558789 samples, validate on 558789 samples
Epoch 1/50
558789/558789 [==============================] - 157s 281us/step - loss: 0.2804 - val_loss: 0.2789

Epoch 00001: val_loss improved from inf to 0.27894, saving model to LSTM_best.hdf5
Epoch 2/50
558789/558789 [==============================] - 152s 273us/step - loss: 0.2802 - val_loss: 0.2789

Epoch 00002: val_loss improved from 0.27894 to 0.27889, saving model to LSTM_best.hdf5
Epoch 3/50
558789/558789 [==============================] - 153s 273us/step - loss: 0.2802 - val_loss: 0.2786

Epoch 00003: val_loss improved from 0.27889 to 0.27858, saving model to LSTM_best.hdf5
Epoch 4/50
558789/558789 [==============================] - 152s 272us/step - loss: 0.2800 - val_loss: 0.2786

Epoch 00004: val_loss improved from 0.27858 to 0.27856, saving model to LSTM_best.hdf5
Epoch 5/50
558789/558789 [==============================] - 153s 273us/step - loss: 0.2799 - val_loss: 0.2784

Epoch 00005: val_loss improved from 0.27856 to 0.

558789/558789 [==============================] - 150s 269us/step - loss: 0.2780 - val_loss: 0.2764

Epoch 00045: val_loss did not improve from 0.27628
Epoch 46/50
558789/558789 [==============================] - 150s 269us/step - loss: 0.2779 - val_loss: 0.2761

Epoch 00046: val_loss improved from 0.27628 to 0.27612, saving model to LSTM_best.hdf5
Epoch 47/50
558789/558789 [==============================] - 150s 269us/step - loss: 0.2779 - val_loss: 0.2762

Epoch 00047: val_loss did not improve from 0.27612
Epoch 48/50
558789/558789 [==============================] - 150s 269us/step - loss: 0.2779 - val_loss: 0.2762

Epoch 00048: val_loss did not improve from 0.27612
Epoch 49/50
558789/558789 [==============================] - 150s 269us/step - loss: 0.2779 - val_loss: 0.2764

Epoch 00049: val_loss did not improve from 0.27612
Epoch 50/50
558789/558789 [==============================] - 150s 269us/step - loss: 0.2779 - val_loss: 0.2763

Epoch 00050: val_loss did not improve from 0.2761

## 文本预测

In [65]:
def predict_next(input_array):
    x = np.reshape(input_array, (-1, seq_length, 100))
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    input_stream = []
    for word in jieba.lcut(raw_input):
        if word in vocab:
            input_stream.append(word)
    res = []
    for word in input_stream[(len(input_stream) - seq_length):]:
        res.append(w2v_model.wv[word])
    return res

def y_to_word(y):
    word = w2v_model.wv.most_similar(positive = y, topn = 1)
    return word

In [76]:
def generate_article(init, rounds = 10):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_word(predict_next(string_to_index(in_string)))
        in_string += n[0][0]
    return in_string

In [78]:
init = '杨过道：“靖康年间徽钦二帝被金兵掳去这件大耻，郭啸天、杨铁心越听越怒。'
article = generate_article(init)
print(article)

杨过道：“靖康年间徽钦二帝被金兵掳去这件大耻，郭啸天、杨铁心越听越怒。哼快逃别生气黄蓉喜活不成赌气小朋友先赌气活不成
